---
title: Data Analysis Projekt Klimaerwärmung
description: Analyse der Temperaturdaten der Erde im Zeitraum von 1750 bis 2015.
show-code: True
show-prompt: True
---

# Gliederung
1. [Import verwendeter Bibliotheken](#section_d)
2. [Einleitung](#section_a)
3. [Globale Temperaturen: allgemein](#section_b)
    1. [Beschreibung des Datensatzes](#section_c)
    2. [Boxplots](#section_e)
    3. [Boxplots pro Monat](#section_f)
    4. [Liniendiagramm](#section_g)
    5. [Liniendiagramm mit minimalen und maximalen Werten](#section_h)
    6. [Regressionsgerade der Durchschnittstemperatur](#section_i)
    7. [Durchschnittstemperaturen im Vergleich zu Messfehlern](#section_j)
    8. [Durchschnittstemperatur pro Jahreszeit](#section_k)
    9. [Boxplot vor und nach 1970](#section_l)
    10. [Regressionsgerade der Durchschnittstemperatur vor und nach 1970](#section_m)
    11. [Durchschnittstemperaturen der Monate im Messzeitraum](#section_y)
    12. [Boxplots zu jedem Jahr des Messzeitraums](#section_x)
    13. [Heatmap der Durchschnittstemperaturen](#section_n)
    14. [Pairplot der Durchschnittstemperaturen](#section_o)
    15. [Saisonale Dekomposition](#section_z)
    16. [Algorithmus zur Datenvorhersage](#section_p)
4. [Globale Temperaturen: Länder](#section_aa)
    1. [Beschreibung des Datensatzes](#section_ab)
    2. [Durchschnittstemperaturen als Weltkugel](#section_ac)
    3. [Durchschnittstemperaturen im Zeitverlauf als Weltkugel](#section_ad)
    4. [Temperaturen pro Kontinent](#section_ae)
    5. [Highest BIP Countrys](#section_af)
    6. [Highest BIP Countrys Regressionsgeraden](#section_ag)
    7. [Lowest BIP Countrys](#section_ah)
    8. [Lowest BIP Countrys Regressionsgeraden](#section_ai)
5. [Verlauf der CO2-Werte ](#section_ba)
    1. [Beschreibung des Datensatzes](#section_bb)
    2. [Korrelationskoeffizient zwischen "GlobalLandTemperatures" und "GlobalCo2Values"](#section_bc)
    3. [Pairplot von "GlobalTemperatures" und "GlobalCo2Values"](#section_bd)
    4. [3D Scatterplot mit LandAverageTemperature, Carbon Dioxide (ppm) und Year](#section_be)
6. [Fazit ](#section_c)
7. [Dataset Quellen](#section_d)
8. [Quellverzeichnis](#section_e)


In [3]:
# Erforderliche Bibliotheken mit pip installieren
# pip install jupyter, pmdarima, pydlm, seaborn, numpy, plotly, prophet

from pathlib import Path
from statsmodels.tsa.seasonal import seasonal_decompose
from prophet import Prophet
from prophet.diagnostics import cross_validation,performance_metrics
from prophet.plot import plot_cross_validation_metric

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import itertools
#import os
#import shutil


%matplotlib inline

Die oben genannten Bibliotheken müssen installiert werden, um das Programm erfolgreich auszuführen.
Dies funktioniert mit pip install jupyter, pmdarima, pydlm, seaborn, numpy, plotly, prophet.

# <a name="section_a">Einleitung </a>
Hitzerekorde, Waldbrände und Hungersnöte.
All das soll durch die menschengemachte Erwärmung des Klimas verursacht werden.
Hierzu lassen sich einige Hypothesen aufstellen, welche in der nachfolgenden Arbeit überprüft werden sollen.
Gibt es die Klimaerwärmung wirklich?
Hat die Industrialisierung einen Einfluss auf die Klimaerwärmung?
Wie wird sich der Temperaturverlauf bis zum Jahr 2030 entwickeln?
Laut Definition beschreibt die Klimaerwärmung die Erwärmung des Klimas auf der Erde über einen langen Zeitraum. (vgl. Quelle 8)
Die Klimaerwärmung ist Teil des Klimawandels. Neben ihr gibt es noch weitere Einflussfaktoren auf den Klimawandel wie Niederschläge und Meeresströmungen.
In vorliegender Arbeit wird sich jedoch nur auf die Entwicklung der Erdtemperatur bezogen und die aufgestellten Hypothesen mithilfe verschiedener Datensätze überprüft.

# <a name="section_b"> Globale Temperaturen: allgemein </a>

## <a name="section_c"> Beschreibung des Datensatzes </a>
Im Datensatz "GlobalTemperatures" wird der Temperaturverlauf der Erde dargestellt. Hierbei geht es jeweils um die weltweite Durchschnittstemperatur an den einzelnen Tagen vom 01.01.1750 bis zum 12.01.2015. Für jeden einzelnen Tag werden verschiedene Werte betrachtet.
Dazu zählt:
LandAverageTemperatur - Weltweite Durchschnittstemperatur des Tages unter Beachtung der Landtemperatur
LandAverageTemperatureUncertainty - Messfehler des jeweiligen Tages
LandMaxTemperature - Maximale weltweite Durchschnittstemperatur des Tages unter Beachtung der Landtemperatur
LandMaxTemperatureUncertainty - Dazugehörige Messfehler
LandMinTemperature - Minimale weltweite Durchschnittstemperatur des Tages unter Beachtung der Landtemperatur
LandMinTemperatureUncertainty - Dazugehörige Messfehler
LandAndOceanAverageTemperature - Weltweite Durchschnittstemperatur des Tages unter Beachtung der Land- und Ozeantemperatur
LandAndOceanAverageTemperatureUncertainty - Dazugehörige Messfehler
Insgesamt enthält der Datensatz 3192 Datensätze mit insgesamt 9 Attributen. Die Attribute werden für jedes einzelne Datum im angegebenen Zeitraum ausgegeben.

In [ ]:
globalTemperatures = pd.read_csv(Path().joinpath('data', 'GlobalTemperatures.csv')) #Einlesen von GlobalTemperatures.csv
globalTemperaturesNeural = globalTemperatures.copy() #Unverändertes Dataset für neurales lernen

globalTemperatures.head()

In [ ]:
globalTemperatures.rename(columns={'dt' : 'Date'},inplace=True) #Name "dt" wird Auf "Datum geändert
globalTemperatures['Date'] = pd.to_datetime(globalTemperatures['Date']) #Änderung von Datum auf DateTime Format
globalTemperatures.set_index('Date',inplace = True) #setzen des Datums als Index
globalTemperatures.isnull().sum()


Vorliegende Übersicht zeigt die Anzahl der null-Werte in der einzelnen Attribute.
Alle unvollständigen Datensätze werden nicht mitbewertet.
Insgesamt handelt es sich dabei um 1200 Datensätze, welche nicht mitbewertet werden können. Somit sind ca. 38% der Daten des Datensatzes "GlobalTemperatures" unbrauchbar.
Gearbeitet wird mit 1992 Datensätzen mit jeweils 9 Attributen.
Nachfolgend wird zur besseren Orientierung des Datensatzes eine Tabelle mit allen enthaltenen Spalten und 10 Zeilen des Datensatzes ausgegeben.

In [ ]:
globalTemperatures = globalTemperatures.dropna(how='any' ,axis=0)
globalTemperatures.head(10)

## <a name="section_e"> Boxplots </a>


In [ ]:
f, allValuesBoxplot = plt.subplots(2,4 , figsize = (15,10))
allValuesBoxplot[0,0].boxplot(globalTemperatures['LandAverageTemperature'])
allValuesBoxplot[0,0].set_title("LandAverageTemp")

allValuesBoxplot[0,1].boxplot(globalTemperatures['LandAverageTemperatureUncertainty'])
allValuesBoxplot[0,1].set_title("LandAverageTempUncertainty")

allValuesBoxplot[0,2].boxplot(globalTemperatures['LandMaxTemperature'])
allValuesBoxplot[0,2].set_title("LandMaxTemp")

allValuesBoxplot[0,3].boxplot(globalTemperatures['LandMaxTemperatureUncertainty'])
allValuesBoxplot[0,3].set_title("LandMaxTempUncertainty")

allValuesBoxplot[1,0].boxplot(globalTemperatures['LandMinTemperature'])
allValuesBoxplot[1,0].set_title("LandMinTemp")

allValuesBoxplot[1,1].boxplot(globalTemperatures['LandMinTemperatureUncertainty'])
allValuesBoxplot[1,1].set_title("LandMinTempUncertainty")

allValuesBoxplot[1,2].boxplot(globalTemperatures['LandAndOceanAverageTemperature'])
allValuesBoxplot[1,2].set_title("Land/OceanAverageTemp")

allValuesBoxplot[1,3].boxplot(globalTemperatures['LandAndOceanAverageTemperatureUncertainty'])
allValuesBoxplot[1,3].set_title("Land/OceanAverageTempeUncertainty")

plt.savefig(r'plots\(01) globalTempsAllValuesBoxplot')

Zu jedem Attribut des Datensatzes "GlobalTemperatures" wurde ein Boxplot, auch Box-Whisker-Plot genannt, generiert.
Ein Boxplot dient der übersichtlichen Darstellung der wichtigsten Lage- und Streuungsmaße. In einem Boxplot werden der Median, das Minimum (unterer Whisker), das Maximum (oberer Whisker), sowie das untere Quartil und das obere Quartil dargestellt. Zudem gibt es in einem Boxplot noch Ausreißer. Diese werden als einzelne Punkte außerhalb der Whisker dargestellt.
Das Minimum eines Boxplots beschreibt den kleinsten Wert. Das Maximum dagegen den größten Wert der Daten. Sie werden durch die Whisker unterhalb und oberhalb der Quartile dargestellt.
 Der Median ist der Wert, welcher sich genau in der Mitte der Daten befindet und ist im Gegensatz zum Mittelwert nicht der Durchschnitt aus allen Werten. Der Median wird durch einen Strich gekennzeichnet welcher sich meist ca. in der Mitte des durch die Quartile begrenzten Kastens befindet. Das untere Quartil beschreibt den Wert der über 25% der Werte liegt. Das obere Quartil beschreibt dagegen der Wert der über 75% der Werte liegt. Ausreißer sind Werte die sich nicht innerhalb der Spannweite des oberen und unteren Whiskers befinden.
In den vorliegenden Boxplots ist besonders auffällig, dass die Boxplots, welche die Werte der Messfehler darstellen, besonders viele Ausreißer haben. Dies könnte daran liegen, dass sich die Großzahl der Messfehler in einem sehr kleinen Zahlenbereich befinden. Sobald ein Messfehler eine etwas größere Abweichung hat, befindet er sich schnell außerhalb dieses Bereiches. Schaut man sich die Werte der Messfehler im Datensatz an, fällt auf, dass die Messfehler mit der Zeit immer weiter abnehmen. Zu Beginn der Aufzeichnungen liegt der Messfehler teilweise über 5. Zum Ende der Aufzeichnungen liegt ein Großteil der Messfehler unter 1. Diese Veränderung lässt sich auf die kontinuierliche Verbesserung von Messverfahren und Messtechniken, welche sich mit der Zeit entwickelt haben, zurückführen.

## <a name="section_f"> Boxplots pro Monat </a>

In [ ]:
globalTemperatures["Month"]= globalTemperatures.index.month
sns.set(rc = {'figure.figsize':(15,8)})
sns.boxplot(x="Month",
            y="LandAverageTemperature",
            data=globalTemperatures).set(title='Durchschnitsstemperaturen gesamt Boxplot')



plt.savefig(r'plots\(02) plotsPerMonth')

In vorliegender Grafik ist ein Diagramm dargestellt, welches aus Boxplots besteht, die aus den Durschnittstemperaturen der Länder in den einzelnen Monaten erstellt wurden. Man sieht, dass die Temperatur zu Beginn des Jahres ansteigt und nach ihrem Höhepunkt im Juli wieder deutlich abfällt. Auffällig ist, dass in den Wintermonaten kaum Ausreißer zu erkennen sind. In den Sommermonaten von Mai bis Oktober gibt es hingegen einige Ausreißer. Dies könnte sich dadurch begründen, dass direkte Sonneneinstrahlung die größte Fehlerquelle beim Messen von Temperaturen darstellt. Dieses Problem entsteht aufgrund der Funktionsweise von Thermometern. Ein Thermometer funktioniert mit Flüssigkeit, welche sich an die Umgebungstemperatur anpasst. Das Thermometer misst dann seine eigene Temperatur (vgl. Quelle 1). Wird ein Thermometer nun in der Sonne platziert, ist die Temperatur deutlich höher als die eigentliche Umgebungstemperatur.
Da die Anzahl Sonnenstunden im Sommer deutlich höher und die Stärke der Sonnenstrahlen stärker sind als im Winter, lässt sich die erhöhte Anzahl an Messfehler vermutlich darauf zurückführen. Dies lässt sich auch dadurch begründen, dass die Ausreißer hauptsächlich überhalb des oberen Whiskers liegen und es sich somit, um Messfehler handeln muss, welche durch eine zu hoch gemessene Temperatur entstanden sind.

## <a name="section_g"> Liniendiagramm </a>

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.lineplot(x="Month",
             y="LandAverageTemperature",
             data=globalTemperatures,
             label = "LandAverageTemperature").set(title='Durchschnitsstemperaturen gesamt') #Titel

sns.lineplot(x="Month",
             y= "LandAndOceanAverageTemperature",
             data= globalTemperatures,
             label = "LandAndOceanAverageTemperature")

sns.lineplot(x="Month",
             y= "LandMinTemperature",
             data= globalTemperatures,
             label = "LandMinTemperature")

sns.lineplot(x="Month",
             y= "LandMaxTemperature",
             data= globalTemperatures,
             label = "LandMaxTemperature")

plt.legend()
plt.savefig(r'plots\(03) plotsPerMonthLines')

In dieser Grafik wurde für die vier Attribute "LandAverageTemperature", "LandAndOceanTemperature", "LandMinTemperature" und "LandMaxTemperature" des Datensatzes "GlobalTemperatures" ein Liniendiagramm erstellt. Die drei Attribute "LandMinTemperature", "LandMaxTemperature" und "LandAverageTemperature" haben einen sehr ähnlichen Verlauf und unterschieden sich jeweils um ca. 5° Celsius. Die "LandAndOceanTemperature" verläuft als einzige anders. Ihr Temperaturverlauf ist deutlich gleichmäßiger und unterliegt insgesamt nur sehr geringen Temperaturschwankungen. Daraus lässt sich schließen, dass die Meerestemperatur über das Jahr deutlich geringeren Temperaturschwankungen unterliegt als die Lufttemperatur und die Temperaturschwankungen auf dem Land damit ausgleicht.
Die Wassertemperatur von Ozeanen ist abhängig von der Außentemperatur. Es erwärmt sich jedoch lediglich das Wasser an der Oberfläche, da die Sonnenstrahlen aufgrund der tiefe des Meeres nur bis in einen bestimmten Bereich vordringen können. In tieferen Bereichen des Meeres fällt die Wassertemperatur zunehmen ab. (vgl. Quelle 2) Dadurch bleiben die Temperaturen der Meere insgesamt über das Jahr relativ konstant. Die Temperatur des Ozeans hängt zudem vom Breitengrad ab, dies ist hier jedoch nebensächlich, da die Daten der Ozeane in allen verschiedenen Breitengraden gewertet werden.

## <a name="section_h"> Liniendiagramm mit minimalen und maximalen Werten </a>

In [ ]:
globalTemperatures["Year"]=globalTemperatures.index.year
sns.lineplot(x="Year",
             y="LandAverageTemperature",
             data=globalTemperatures,
             label = "LandAverageTemperature").set(title = 'Durchschnittstemperaturen 1850 - 2015')

sns.lineplot(x="Year",
             y="LandAndOceanAverageTemperature",
             data=globalTemperatures,
             label = "LandAndOceanAverageTemperature")

sns.lineplot(x="Year",
             y="LandMinTemperature",
             data=globalTemperatures,
             label = "LandMinTemperature")

sns.lineplot(x="Year",
             y="LandMaxTemperature",
             data=globalTemperatures,
             label = "LandMaxTemperature")

sns.set(rc = {'figure.figsize':(15,8)})

plt.legend()
plt.savefig(r'plots\(04) allAverageTemps')

In dieser Grafik sieht man den Temperaturverlauf der Durchschnittstemperaturen der vier Attribute "LandAverageTemperature", "LandAndOceanTemperature", "LandMinTemperature" und "LandMaxTemperature" über den gesamten Messzeitraum. Dieser beginnt im Jahr 1850 und endet in 2015. Bei allen Attributen ist ein konstanter Temperaturanstieg sichtbar. Neben dem konstanten Anstieg ist im Jahr 1875-1880 ein starker Ausreißer nach oben sichtbar. Dies könnte auf eine zufällig aufgetretene warme Periode deuten. Stattdessen könnte es sich jedoch auch um eine Folge verschiedener historischer Ereignisse handeln, welche einen Einfluss auf die Erdtemperatur haben könnten. Nach der Gründung des Deutschen Reiches im Jahre 1871 kam es zu einer Vielzahl neuer Erfindungen im Bereich Elektrotechnik und Chemie. Zudem wurde Großbritannien zum wichtigsten Exportmarkt für die deutsche Industrie. Es ist jedoch eher davon auszugehen, dass es sich um eine zufällig aufgetretene warme Periode handelt, da industrielle Entwicklungen eher langfristige Auswirkungen haben und nicht in einem einzelnen konkreten Jahr sichtbar werden. Das kann man daraus schließen, dass die Steigung des Temperaturverlaufs nach der zweiten Industriellen Revolution im Jahr 1920 und der dritten Revolution im Jahr 1970 langfristig angestiegen ist und sich nicht in einem einzelnen konkreten Jahr widergespiegelt hat.

## <a name="section_i"> Regressionsgerade der Durchschnittstemperatur </a>

In [ ]:
sns.regplot(x="Year",
            y="LandAverageTemperature",
            data=globalTemperatures).set(title = 'LandAverageTemperature mit Regeressionsgerade')
plt.savefig(r'plots\(05) LandAverageWithRegeression')

In vorliegender Grafik wurde aus den Werten der "LandAverageTemperature" eine Regressionsgerade gebildet. Mithilfe einer Regressionsgerade lässt sich eine lineare Regression bildlich darstellen (vgl. Quelle 3). Eine lineare Regression beschreibt den Zusammenhang zweier Variablen, welche auf der x-Achse und der y-Achse dargestellt werden. Handelt es sich bei der Regressionsgerade um eine ansteigende Gerade, dann besteht ein positiver Zusammenhang zwischen den Variablen. Handelt es sich um eine abfallende Gerade, dann besteht ein negativer Zusammenhang zwischen den Variablen. Bei vorliegender Grafik soll der Zusammenhang zwischen der "LandAverageTemperature" und dem Jahr mithilfe einer Regressionsgerade dargestellt werden. Es ist deutlich zu sehen, dass es sich bei der Regressionsgerade um eine ansteigende Gerade handelt. Daraus lässt sich schließen, dass zwischen der "LandAverageTemperatur" und dem Jahr ein positiver Zusammenhang besteht. Dies bedeutet, dass die durchschnittliche "LandAverageTemperatur" immer weiter ansteigt, je weiter wir uns ín der Zeit bewegen. Vergleicht man den ersten und den letzten Wert der Geraden, sieht man einen Anstieg >1° Celsius.
In der Grafik werden neben der Regressionsgerade auch die einzelnen Werte der "LandAverageTemperature" dargestellt. Es fällt auf, dass es fünf Bereiche gibt in denen die einzelnen Werte angesiedelt sind. Die Bereiche sind Linienförmig angeordnet. Man sieht, dass jeder einzelne dieser Bereiche im Laufe der Zeit ansteigt.
Da die Erde 5 Klimazonen hat, lässt sich darauf schließen, dass die Werte der fünf Bereiche den Werten aus den unterschiedlichen Klimazonen zuzuordnen sind.
Die Steigung der Regressionsgerade ist ein Indiz dafür, dass die These "Gibt es die Klimaerwärmung wirklich?" wahr ist. Durch die Anordnung der Werte in die verschiedenen Klimazonen lässt sich ebenso bestätigen, dass die Erderwärmung ein weltweites Problem ist und sich nicht auf einen bestimmten Bereich beschränkt.

## <a name="section_j"> Durchschnittstemperaturen im Vergleich zu Messfehlern </a>

In [ ]:
sns.lineplot(x="Year",
             y="LandAverageTemperature",
             data=globalTemperatures,
             label = "LandAverageTemperature").set(title = 'Durchschnittstemperaturen ab 1850 Vergleich zu Messfehlern')

sns.lineplot(x="Year",
             y="LandAverageTemperatureUncertainty",
             data=globalTemperatures,
             label = "LandAverageTemperatureError")

plt.legend()
plt.savefig(r'plots\(06) LandAverageAndUncertainty')

In dieser Grafik sind zwei Graphen zu sehen. Der obere Graph zeigt den Temperaturverlauf der durchschnittlichen weltweiten Landtemperatur über die Jahre 1850 bis 2017. Der untere Graph zeigt die durchschnittliche Abweichung durch Messfehler über die Jahre von 1850 bis 2017.
Es wird deutlich, dass die Abweichung durch Messfehler immer weiter abnimmt und im Jahre 2000 gegen null geht. Dies kommt daher, dass sich die Messgeräte über die Zeit stetig verbessert haben, wodurch immer weniger Messfehler zustande kamen. Die Messfehler lagen zu Beginn der Messungen im Durchschnitt bei ca. 1° Celsius. Betrachtet man den Verlauf des Graphen der Durchschnittstemperaturen über die Zeit sieht man, dass der maximale Wert zum Ende der Messungen erreicht wurde. Aufgrund dessen kann man davon ausgehen, dass die Temperatur im weiteren Zeitverlauf noch weiter angestiegen ist. Im Vergleich zum Beginn der Messungen liegt die Temperatur zu Ende der Messungen ca. 2° Celsius höher. Betrachtet man nun den Messfehler, welcher bei ca. 1° Celsius liegt, besteht das Risiko, dass die Werte zu Beginn der Messungen nicht vollständig korrekt sind. Geht man davon aus, dass die realen Werte bereits 1850 1° Celsius über den hier dargestellten Werten lagen, gäbe es im Vergleich zum Jahre 2017 immer noch einen Temperaturanstieg um 1° Celsius. Somit lässt sich ausschließen, dass der weltweite Temperaturanstieg nur durch die damals höhere Messfehlerrate zustande kommt.

## <a name="section_x"> Boxplots zu jedem Jahr des Messzeitraums </a>

In [ ]:
sns.boxplot(x="Year",y="LandAverageTemperature",data=globalTemperatures).set(title = 'Boxplot every Year')
plt.savefig(r'plots\(07) boxplotEveryYear')

In dieser Grafik sieht man für jedes Jahr des Attributs „LandAverageTemperature“ des Datensatzes „GlobalTemperatures“ einen Boxplot. Betrachtet man die unteren Whisker fällt auf, dass im Laufe der Jahre ein deutlicher Anstieg sichtbar ist. Das gleiche gilt für die Mediane sowie die oberen Whisker. Bei den unteren Whiskern ist besonders auffällig, dass es zu Beginn der Messungen besonders häufig Jahre mit sehr niedrigen liegenden Minimalwerten gab. Dies ändert sich im Laufe der Zeit und die minimalen Werte der Boxplots steigen immer weiter an.

## <a name="section_k"> Durchschnittstemperatur pro Jahreszeit </a>

In [ ]:
def get_season(month):
    if 3 <= month <= 5:
        return 'spring'
    elif 6 <= month <= 8:
        return 'summer'
    elif 9 <= month <= 11:
        return 'autumn'
    else:
        return 'winter'

In [ ]:
globalTemperatures['season']=globalTemperatures['Month'].apply(get_season)
years=globalTemperatures['Year'].unique()

spring_temp = []
summer_temp = []
autumn_temp = []
winter_temp = []
for year in years:
    current_df=globalTemperatures[globalTemperatures['Year'] == year]
    spring_temp.append(current_df[current_df['season'] == 'spring']['LandAverageTemperature'].mean())
    summer_temp.append(current_df[current_df['season'] == 'summer']['LandAverageTemperature'].mean())
    autumn_temp.append(current_df[current_df['season'] == 'autumn']['LandAverageTemperature'].mean())
    winter_temp.append(current_df[current_df['season'] == 'winter']['LandAverageTemperature'].mean())

In [ ]:
f, ax = plt.subplots(figsize=(15,8))
plt.plot(years,
         summer_temp,
         label='Summers average temperature',
         color='yellow')

plt.plot(years,
         autumn_temp,
         label='Autumns average temperature',
         color='brown')
plt.plot(years,
         spring_temp,
         label='Springs average temperature',
         color='pink')
plt.plot(years,
         winter_temp,
         label='Winters average temperature',
         color='blue')

ax.set_ylabel('LandAverageTemperature')
ax.set_xlabel('Year')
ax.set_title('Durchschnittstemperatur pro Jahreszeit')
plt.legend()
plt.savefig(r'plots\(08) seasonPlot')

In dieser Grafik sieht man den durchschnittlichen Temperaturverlauf der einzelnen Jahreszeiten über den gesamten Messzeitraum. Frühling sind hierbei die Monate März, April und Mai, Sommer die Monate Juni, Juli und August, Herbst die Monate September, Oktober und November und Winter die Monate Dezember, Januar und Februar. Die Verläufe der einzelnen Jahreszeiten sind relativ ähnlich. Sie sind lediglich auf der y-Achse stark verschoben. Der Größte durschnittliche Temperaturunterschied liegt zwischen den Monaten Sommer und Winter, Frühling und Herbst liegen dagegen sehr nah beieinander. Besonders auffällig ist, dass in allen vier Graphen eine leichte Steigung zu verzeichnen ist. Die Steigung wird ab dem Jahr 1975 deutlich sichtbar steiler als in den Jahren zuvor. Auch dies könnte ein Indiz dafür sein, dass die dritte Industrielle Revolution im Jahre 1970 einen großen Einfluss auf den weltweiten Temperaturverlauf hat.

## <a name="section_l"> Boxplot vor und nach 1970 </a>

In [ ]:
tempsBeforeAndAfter = globalTemperatures.groupby(by = 'Year')[['LandAverageTemperature', 'LandAverageTemperatureUncertainty',
       'LandMaxTemperature', 'LandMaxTemperatureUncertainty',
       'LandMinTemperature', 'LandMinTemperatureUncertainty',
       'LandAndOceanAverageTemperature',
       'LandAndOceanAverageTemperatureUncertainty']].mean().reset_index()

tempsBeforeAndAfter['turnpoint'] = np.where(tempsBeforeAndAfter['Year'] <= 1975, 'before', 'after')

sns.boxplot(x="LandAverageTemperature", y="turnpoint", data=tempsBeforeAndAfter).set(title = 'Durchschnittstemperatur vor und nach 1970')
sns.swarmplot(x="LandAverageTemperature", y="turnpoint", data=tempsBeforeAndAfter, color = "black")

plt.savefig(r'plots\(09) boxplotBeforeAndAfter1970')

In vorliegender Grafik wurde mit Boxplots gearbeitet. Diese unterscheiden sich insofern zu den Boxplots aus vorangegangen Grafiken, da sie nicht vertikal, sondern horizontal aufgebaut sind. Der Aufbau des Boxplots ist immer noch der gleiche. Das Minimum wird nun durch den linken Whisker dargestellt und das Maximum durch den rechten Whisker.
Die Boxplots zeigen die Durchschnittstemperaturen vor und nach dem Jahr 1970. Es fällt besonders auf, dass sich der gesamte Boxplot mit den Werten vor 1970 deutlich weiter links befindet als der Boxplot mit den Werten nach 1970. Dies bedeutet, dass die Temperaturen in den Jahren vor der dritten industriellen Revolution insgesamt deutlich geringer waren als in den Jahren danach.
Es ist zudem deutlich zu erkennen, dass in dem Boxplot mit den Temperaturen der Jahre vor 1970 deutlich mehr Ausreißer enthalten sind als in den Jahren danach. Dies liegt vermutlich daran, dass sich Messtechniken über die Jahre weiterentwickelt haben und somit in den Jahren nach 1970 deutlich weniger Messfehler gemacht wurden, wodurch folglich weniger Ausreißer entstanden sind.
Der Unterschied zwischen den unteren Whiskern der Boxplots liegt bei ca. 0,8° Celsius. Der Unterschied der oberen Whisker des Boxplots liegt bei ca. 0,9° Celsius. Daraus lässt sich auf eine allgemeine Temperaturveränderung schließen. Dies ist ein weiterer Hinweis darauf, dass die Industrialisierung einen Einfluss auf die Klimaerwärmung der Erde hat.

## <a name="section_m"> Regressionsgerade der Durchschnittstemperatur vor und nach 1970 </a>

In [ ]:
reggression = globalTemperatures[globalTemperatures['Year']<=1970]
sns.regplot(x="Year",
            y="LandAverageTemperature",
            data=reggression,
            label= "Vor 1970").set(title = 'Regression vor und nach 1970')

reggression = globalTemperatures[globalTemperatures['Year']>1970]
sns.regplot(x="Year",
            y="LandAverageTemperature",
            data=reggression,
            label= "Nach 1970")

plt.legend()
plt.savefig(r'plots\(10) regeressionBeforeAndAfter1970')

In dieser Grafik wird eine Regressionsgerade aus den Werten der Durchschnittstemperatur im Messzeitraum erstellt. Der Unterschied zur Grafik "Regressionsgerade der Durchschnittstemperatur" ist jedoch, dass aus den Werten ab 1970 eine seperate Regressionsgerade erstellt wurde. Durch den direkten Vergleich ist deutlich zu sehen, dass die Steigung der Regressionsgerade, welche aus den Werten ab 1970 erstellt wurde deutlich steiler ist als die Steigung der Regressionsgerade die aus den Werten vor 1970 erstellt wurde. Dies ist ebenfalls ein Indiz dafür, dass besonders die dritte industrielle Revolution ab 1970 einen starken Einfluss auf die Klimaerwärmung der Erde hat.

## <a name="section_y"> Durchschnittstemperaturen der Monate im Messzeitraum </a>

In [ ]:
pivotTable = globalTemperatures.pivot_table(values='LandAverageTemperature',
                                            index='Month',
                                            columns='Year')

pivotTable

Diese Tabelle zeigt die Durchschnittstemperaturen der einzelnen Monate in den Jahren 1850 bis zum Ende des Messzeitraums im Jahr 2015. Sie zeigt die gleichen Werte an wie die darauffolgende Heatmap, mit dem Unterschied, dass hier die Werte des gesamten Messzeitraums von 1850-2015 aufgelistet sind. Dagegen sind in der "Heatmap der Durchschnittstemperaturen" nur die Werte ab 1950 dargestellt.

## <a name="section_n"> Heatmap der Durchschnittstemperaturen </a>

In [ ]:
plt.figure(figsize=(30,8))
sns.heatmap(data=pivotTable.loc[:,years[100:]],
            annot=True,
            cmap="coolwarm").set(title = 'Heatmap 1950 - 2015')
plt.savefig(r'plots\(11) heatmap')

In dieser Heatmap sieht man die Temperaturen der einzelnen Monate in den Jahren 1950 bis 2015. Vergleicht man die Jahre 1950 und 2015 für die einzelnen Monate, fällt auf, dass es in jedem Monat eine Temperaturerhöhung von 1-2° Celsius gab. Vergleicht man nun das Jahr 1950 und 1970 sieht man, dass die Temperatur relativ konstant geblieben ist und sich nicht stark verändert hat. Vergleicht man nun jedoch das Jahr 1970 mit dem Jahr 2015 sieht man, dass die Temperatursteigungen hauptsächlich in diesem Zeitraum erfolgt sind und zwischen dem Jahr 1970 und 2015 ein starker Temperaturanstieg von 1-2° Celsius in allen Monaten zu verzeichnen ist. Dies unterstützt die These, dass die 3. industrielle Revolution im Jahr 1970 die Klimaerwärmung deutlich vorangetrieben hat. Auch wenn der Temperaturanstieg hauptsächlich ab dem Jahr 1970 erfolgt, lässt sich die 3. industrielle Revolution nicht als alleinigen Faktor der Erderwärmung bezeichnen. Denn auch die Folgen der 1. und 2. industriellen Revolution tragen einen Teil dazu bei, dass die Temperatur stetig ansteigt.

## <a name="section_o"> Pairplot der Durchschnittstemperaturen </a>

In [ ]:
globalTemperaturesWitUncertaintValuesRemoved = globalTemperatures.drop(['LandAverageTemperatureUncertainty',
                                                                        'LandMaxTemperatureUncertainty',
                                                                        'LandMinTemperatureUncertainty',
                                                                        'LandAndOceanAverageTemperatureUncertainty'], axis=1)

sns.pairplot(globalTemperaturesWitUncertaintValuesRemoved)

plt.savefig(r'plots\(12) pairPlotGlobalTemps')

Bei dieser Grafik handelt es sich um einen Pairplot. Bei einem Pairplot werden die Korrelationen zwischen den einzelnen Variablen des Datensatzes dargestellt. Neben den Attributen des Datensatzes werden in diesem Pairplot noch die Monate und die Jahre miteinbezogen. Bei der Interpretation wird sich nun nur auf diejenigen Grafiken bezogen, bei welchen eine Korrelation erkennbar ist.
 Die "LandAverageTemperature" hat sowohl mit der "LandMaxTemperature", als auch mit der "LandMinTemperature" und der "LandAndOceanAverageTemperature" eine starke Korrelation. Die Korrelation zeigt sich in Form einer deutlichen Steigung. Dies bedeutet, dass je höher die "LandAverageTemperature" ist, desto höher sind auch "LandMaxTemperature", "LandMinTemperature" und die "LandAndOceanAverageTemperature". Zwischen jedem der vier genannten Attribute besteht eine starke Korrelation.

## <a name="section_z"> Saisonale Dekomposition </a>

In [ ]:
dataCollection = seasonal_decompose(globalTemperatures["LandAverageTemperature"],model="additive")
dataCollection.plot();
plt.savefig(r'plots\(13) decomposePlot')

In vorliegender Grafik wird eine Dekomposition durchgeführt. Es werden die drei Werte Trend, Seasonal und Resid als jeweils separater Graph angezeigt, welche sich jeweils unterschiedlich interpretieren lassen.
Die erste Grafik zeigt die saisonalen Schwankungen des Attributs "LandAverageTemperature" des Datensatzes "GlobalTemperatures" an.
Die Grafik Trend zeigt die Entwicklung des Durchschnittswertes der Daten der "LandAverageTemperature" des Datensatzes "GlobalTemperatures". In dieser Grafik sieht man, dass der Trend deutlich nach oben geht. Die Durchschnittstemperatur liegt in den ersten Jahren der Aufzeichnungen bei 8° Celsius und teilweise sogar darunter. Über die Jahre steigt die Durchschnittstemperatur immer weiter und liegt am Ende weit über 9° Celsius und erreicht fast die 10° Celsius. Dies würde ein Anstieg um 2° Celsius bedeuten. Schaut man sich den Gesamtverlauf an, ist ein weiterer Anstieg der Durchschnittstemperatur zu erwarten.
Die Grafik Residual zeigt den Unterschied zwischen dem erwarteten und dem tatsächlichen Wert.

## <a name="section_p"> Algorithmus zur Datenvorhersage </a>
Mithilfe des Prophet-Algorithmus soll auf Grundlage der vorhandenen Daten der "LandAverageTemperature" des Datensatzes "GlobalTemperatures" eine Vorhersage zur Entwicklung der Temperatur im zukünftigen Zeitverlauf gemacht werden.

In [ ]:
globalTemperaturesNeural.reset_index(inplace=True)
globalTemperaturesNeural.rename(columns={"dt":"ds","LandAverageTemperature":"y"},inplace=True)
globalTemperaturesNeural=globalTemperaturesNeural[["ds","y"]]

paramGrid={"seasonality_prior_scale":[0.001,0.01,0.1,0.5],
            "changepoint_prior_scale":[0.001,0.01,0.1,10]}

allParams=[dict(zip(paramGrid.keys(),v)) for v in itertools.product(*paramGrid.values())]

rmse= []

for params in allParams:
    model=Prophet(weekly_seasonality=False,
                  daily_seasonality=False,
                  **params)

    model.add_seasonality(name="monthly",
                          period=30.5,
                          fourier_order=5)

    model.fit(globalTemperaturesNeural)

    cv=cross_validation(model,initial="36500 days",
                        horizon="365 days",
                        parallel="processes")

    pm=performance_metrics(cv)
    rmse.append(pm["rmse"].values[0])

best_params=allParams[np.argmin(rmse)]

In [ ]:
fig = plot_cross_validation_metric(cv, metric='rmse')

plt.savefig(r'plots\(14) validationMetric')

In dieser Grafik wird der Root Mean Squared Error (RMSE) für die Datenvorhersage bestimmter Zeiträume angegeben. Der Root Mean Squared Error wird auf der y-Achse angezeigt und wird in Prozent angegeben. Die 1 steht hierbei für 100%. Auf der x-Achse wird der Zeitraum der Vorhersage angegeben. Die Grafik soll zeigen wie hoch die Abweichung der Vorhersage vom tatsächlichen Wert ist, wenn man den Wert für einen bestimmten Zeitraum vorhersagt. Die grauen Punkte zeigen den absoluten Fehler in Prozent. Die blaue Linie zeigt den RMSE. Um den Root Mean Squared Error zu berechnen werden die einzelnen Fehler quadriert und summiert. Dieses Ergebnis wird durch die Gesamtzahl der Fehler dividiert und daraus die Quadratwurzel gezogen. (Vgl. Quelle 7). Man sieht, dass der prozentuale Fehler steigt, je weiter die Voraussage in der Zukunft liegt.

In [ ]:
model=Prophet(weekly_seasonality=False,
              daily_seasonality=False,
              **best_params)

model.add_seasonality(name="monthly",
                      period=30.5,
                      fourier_order=6)

model.fit(globalTemperaturesNeural)

In [ ]:
future=model.make_future_dataframe(periods=220,
                                   freq="MS")

forecast=model.predict(future)
model.plot(forecast);
plt.savefig(r'plots\(15) foreCastModel')

Mithilfe dieser Regressionsfläche wird der Verlauf der Daten im Messzeitraum angegeben. Darüber hinaus wird der weitere Verlauf des Graphen bis zum Jahr 2030 grafisch dargestellt. Die Punkte im Diagramm sind die einzelnen Werte des Attributs "LandAverageTemperature". Da die Messwerte des Datensatzes nur bis zum Jahr 2015 vorhanden sind, sind ab diesem Zeitpunkt keine Punkte mehr vorhanden, sondern lediglich die Regressionsfläche, welche ab diesem Zeitpunkt nicht mehr auf den Daten, sondern auf der Prognose der Werte beruht. Die Regressionsfläche geht bis ca. 2030. Da es sich um eine Fläche handelt, lassen sich keine genauen Werte ablesen. Es ist jedoch deutlich zu sehen, dass die Tendenz stetig steigend ist.

In [ ]:
model.plot_components(forecast);
plt.savefig(r'plots\(16) foreCastComponents')

Vorliegende Grafik beruht ebenfalls auf dem Prophet Algorithmus. Dargestellt sind drei Graphen. Der erste Graph zeigt den Trend über die Jahre an. Im Gegensatz zur vorhergehenden Grafik bildet er aus den Werten der "LandAverageTemperature" einen Durchschnittswert und zeigt auch bei der Prognose einen Durchschnittswert an. Dadurch lässt er sich deutlich besser ablesen. Im Jahr 2030 liegt die erwartete Temperatur knapp über 10° Celsius. Zudem fällt auf, dass zu Beginn der Zeit noch starke Schwankungen nach unten vorhanden waren. Ab ca. 1960 ist jedoch eine lineare Steigung des Graphen zu erkennen. Im Vergleich zum Höchstwert des Jahres 1750 ist die Temperatur bis zum Jahr 2030 somit um über 1° Celsius gestiegen.
In der zweiten Grafik wird eine Prognose zum Temperaturverlauf über das Jahr abgegeben. Deutlich ist, dass die Temperaturen zu den Sommermonaten deutlich ansteigen und ab August wieder deutlich abfallen.
In der dritten Grafik wird der prognostizierte Temperaturverlauf eines Monats dargestellt. Dieser weißt große Schwankungen auf, die Ursache hierfür lässt sich jedoch nicht eindeutig bestimmen.

# <a name="section_aa"> Globale Temperaturen: Länder </a>

## <a name="section_ab"> Beschreibung des Datensatzes </a>
Im Datensatz "GlobalLandTemperaturesByCountry" werden die Durchschnittstemperaturen der einzelnen Länder der Erde für jeden Tag über den Zeitraum vom 11.01.1743 bis zum 09.01.2013 dargestellt.
Für jedes Land werden an einem bestimmten Datum eines bestimmten Jahres verschiedene Werte betrachtet.
Dazu zählt:
AverageTemperatureByCountry - Durchschnittstemperatur des Landes am jeweiligen Datum
AverageTemperatureByCountryUncertainty - Messfehler der einzelnen Werte
Die Spalte, in welcher das Jahr angegeben wird, wurde manuell eingefügt.
Insgesamt enthält der Datensatz 577462 Datensätze mit insgesamt 5 Attributen.

In [ ]:
globalLandTemperaturesByCountry = pd.read_csv(Path().joinpath('data', 'GlobalLandTemperaturesByCountry.csv'))
globalLandTemperaturesByCountry.rename(columns={'dt' : 'Date'},inplace=True)
globalLandTemperaturesByCountry['Date'] = pd.to_datetime(globalLandTemperaturesByCountry['Date'])
globalLandTemperaturesByCountry['Year'] = globalLandTemperaturesByCountry['Date'].dt.year
globalLandTemperaturesByCountry.isnull().sum()

Vorliegende Übersicht zeigt die Anzahl der Null-Werte in den einzelnen Spalten.
Alle unvollständigen Datensätze werden nicht mitbewertet.
Insgesamt handelt es sich dabei um 32651 Datensätze welche nicht mitbewertet werden können. Somit sind ca. 5,7% der Daten des Datensatzes "GlobalLandTemperaturesByCountry" unbrauchbar.
Gearbeitet wird mit 544811 Datensätzen mit jeweils 5 Attributen.

In [ ]:
nasPerDate = globalLandTemperaturesByCountry.set_index('Date')['AverageTemperature']\
    .isnull()\
    .reset_index()\
    .groupby('Date')\
    .sum()

nasPerDate.columns = ['Missing values']
nasPerDate['Year']= nasPerDate.index.year
nasPerDate = nasPerDate[nasPerDate['Year'] <= 2000]

sns.lineplot(x="Date",
             y= "Missing values",
             data= nasPerDate).set(title = 'Fehlende Werte Zeitverlauf')
plt.savefig(r'plots\(17) nasPerDateCountry')

In dieser Grafik werden die Null-Werte über die verschiedenen Jahre dargestellt. Besonders auffällig ist, dass die Zahl der Null-Values um das Jahr 1800 deutlich ansteigt und erst gegen 1880 wieder fällt. Dies kann daher kommen, dass um das Jahr 1800 mehr Länder hinzugekommen sind, bei welchen die Temperaturen gemessen wurden. Gibt es insgesamt mehr Länder, steigt jedoch auch die Gesamtzahl der Null-values in den Messwerten. Die Zahl der Null-values fällt erst um 1880 wieder, da ab diesem Zeitpunkt vermutlich zuverlässiger gemessen werden konnte.

## <a name="section_ac"> Durchschnittstemperaturen als Weltkugel  </a>

In [ ]:
averageTemperatureByCountry= globalLandTemperaturesByCountry\
    .groupby(by ='Country')\
    .mean()

averageTemperatureByCountry= averageTemperatureByCountry.reset_index()

figWorldMap = px.choropleth(averageTemperatureByCountry,
                            locations='Country',
                            locationmode='country names',
                            color='AverageTemperature',
                            color_continuous_scale= 'temps',
                            hover_name='Country',
                            title = 'Globale Temperaturen')

figWorldMap.update_geos(projection_type="orthographic",
                        showocean=True,
                        showlakes=False,
                        showcountries=True,
                        lataxis_showgrid=True,
                        lonaxis_showgrid=True)

figWorldMap.update_layout(height=700,
                          margin={"r":0,"t":50,"l":0,"b":20})
figWorldMap.show()

In vorliegender Grafik sind die weltweiten Durchschnittstemperaturen der einzelnen Länder grafisch dargestellt. Die Farbe des jeweiligen Landes entspricht der Durchschnittstemperatur dieses Landes im Messzeitraum. Die Farbskala beginnt bei blau. Blau stellt die kälteste Temperatur auf der Farbskala dar. Über grün, gelb, orange und schließlich rot, werden immer wärmere Temperaturen dargestellt. Hat ein Land keine der Farbskala entsprechende Farbe, sind für dieses Land keine oder nur unzureichende Messwerte vorhanden.
Anhand dieser Weltkugel lassen sich die fünf Klimazonen Polare Zone, Subpolare Zone, Gemäßigte Zone, Subtropische Zone und Tropische Zone besonders gut veranschaulichen.
Im oberen Teil der Erde, der Polaren Zone, sieht man eine dunkel- bis hellgrüne Verfärbung. An der Farbskala lässt sich ablesen, dass es sich um eine Durchschnittstemperatur zwischen -15°C und -5°C handelt. In der darauffolgenden Subpolaren Zone sind die Länder hauptsächlich in gelb und hellgrün eingefärbt. Daraus lässt sich eine Durchschnittstemperatur von 0-5° Celsius ableiten. Die darauffolgende Gemäßigte Zone ist größtenteils in gelb eingefärbt und hat somit eine Durchschnittstemperatur von 5-10° Celsius. Nach der Gemäßigten Zone folgt die Subtropische Zone, welche sich in der Durchschnittstemperatur bereits an die Tropische Zone annähert. In der Mitte der Erdkugel befindet sich die Tropische Zone. In dieser ist die Durschnittstemperatur im Vergleich am höchsten. Durch die rote Einfärbung lässt sich auf eine Durchschnittstemperatur von 20-25°Celsius schließen. Nach der Tropischen Zone folgt auf der Südhalbkugel, wie auf der Nordhalbkugel, die Subtropische Zone, die Gemäßigte Zone, die Subpolare Zone und die Polare Zone, auf welcher der Südpol liegt. In vorliegender Grafik liegen für den Südpol jedoch keine Werte vor, weshalb er in keiner der Farben der Farbskala eingefärbt ist. (Vgl. Quelle 4)


## <a name="section_ad"> Durchschnittstemperatur als Weltkugel im Zeitverlauf </a>

In [ ]:
figWorldMap = px.choropleth(globalLandTemperaturesByCountry,
                            locations='Country',
                            locationmode='country names', color='AverageTemperature',
                            color_continuous_scale='temps', hover_name='Country',
                            animation_frame='Year', animation_group='Country',
                            title = 'Globale Temperaturen Zeitverlauf',
                            range_color=[globalLandTemperaturesByCountry['AverageTemperature'].min(),
                                         globalLandTemperaturesByCountry['AverageTemperature'].max()])

figWorldMap.update_geos(projection_type="orthographic",
                        showocean = True,
                        showlakes=False,
                        showcountries=True,
                        lataxis_showgrid=True,
                        lonaxis_showgrid=True)
figWorldMap.update_layout(height=700,
                          margin={"r":0,"t":0,"l":0,"b":0})

figWorldMap.show()

Vorliegende Grafik zeigt eine Weltkugel, welche genauso aufgebaut ist wie die Weltkugel in der Grafik darüber. Der Unterschied ist jedoch, dass man sich die Durchschnittstemperaturen jeden Jahres vom Jahr 1743 bis 2007 anzeigen lassen kann. Diesen Prozess kann man über den Play-Button starten. Am Anfang fällt besonders auf, dass den meisten Ländern noch keine Farbe zugeordnet ist. Dies liegt daran, dass zu diesem Zeitpunkt noch keine Messwerte vorhanden sind. Je weiter man in der Zeit voranschreitet, desto mehr Messwerte sind vorhanden, weshalb immer mehr Länder mit der jeweiligen Farbe gekennzeichnet werden. Da die Farbskala zu unpräzise ist, um leichte Temperaturschwankungen mit bloßem Auge zu erkennen, lässt sich anhand der Optik nur teilweise und auf einzelne Länder bezogen ein Unterschied feststellen. Schaut man sich die Temperaturen beispielhaft am Land Algerien an, welches im Jahr 1790 eine Temperatur von 11,94°Celsius und im Jahr 2011 eine Temperatur von 14.015°Celsius hat, ist ein deutlicher Temperaturunterschied zu erkennen. Der Anstieg unterscheidet sich jedoch von Land zu Land und ist in manchen Ländern auch allein anhand der Farbskala zu erkennen.


## <a name="section_ae"> Temperaturen pro Kontinent </a>

In [ ]:
globalLandTemperaturesByContinent = globalLandTemperaturesByCountry.groupby(by = ['Year', 'Country']).mean().reset_index()

regionMapping = pd.read_csv(Path().joinpath('data', 'CountryMapping.csv'))
regionMapping['Country'] = regionMapping['name']
regionMapping = regionMapping[['Country', 'region', 'alpha-2', 'alpha-3']]

data = pd.merge(left = globalLandTemperaturesByContinent,
                right = regionMapping,
                on = 'Country',
                how = 'left')

region = data.dropna(axis = 0).groupby(by = ['region', 'Year']).mean().reset_index()

In [ ]:
europe = region[region['region']=='Europe']
americas = region[region['region']=='Americas']
asia = region[region['region']=='Asia']
africa = region[region['region']=='Africa']
oceania = region[region['region']=='Oceania']

europe = europe.groupby('Year')['AverageTemperature'].mean().reset_index()
americas = americas.groupby('Year')['AverageTemperature'].mean().reset_index()
asia = asia .groupby('Year')['AverageTemperature'].mean().reset_index()
africa = africa.groupby('Year')['AverageTemperature'].mean().reset_index()
oceania = oceania.groupby('Year')['AverageTemperature'].mean().reset_index()

sns.lineplot(x="Year",
             y= "AverageTemperature",
             data= europe,
             label = "Europe",
             color = "black").set(title = 'Temperatur je Kontinent (Amerika = Nord/ + Süd/ Amerika)')

sns.lineplot(x="Year",
             y= "AverageTemperature",
             data= americas,
             label = "Americas",
             color = "green")

sns.lineplot(x="Year",
             y= "AverageTemperature",
             data= asia,
             label = "Asia",
             color = "red")

sns.lineplot(x="Year",
             y= "AverageTemperature",
             data= africa,
             label = "Africa",
             color = "yellow")

sns.lineplot(x="Year",
             y= "AverageTemperature",
             data= oceania,
             label = "Oceania",
             color = "blue")

plt.legend()
plt.savefig(r'plots\(18) kontinentTemps')

In der Grafik "Temperatur je Kontinent" wird für jeden Kontinent ein Graph angezeigt, welcher die Durchschnittstemperatur der Länder auf dem jeweiligen Kontinent über den Messzeitraum darstellt. Es ist jedoch zu beachten, dass für Nord- und Südamerika keine separaten Daten vorhanden sind, weshalb diese als ein Kontinent mit einem gemeinsamen Graphen dargestellt werden.
Es fällt auf, dass die Verläufe der Graphen zu Beginn sehr ungleichmäßig verlaufen. Im Laufe der Zeit wird der Verlauf immer gleichmäßiger und die Graphen der Kontinente ähneln sich stark und sind hauptsächlich auf der y-Achse nach oben oder unten verschoben.
Die starken Schwankungen zu Beginn der Messungen lassen sich auf inkonsistente Messungen zurückführen. Der Graph vom Kontinent Europa verläuft schon seit Beginn an sehr gleichmäßig. Schaut man sich jedoch die Grafik "Durchschnittstemperaturen als Weltkugel im Zeitverlauf" an, dann wird deutlich, dass die europäischen Länder zu den wenigen Ländern gehören, in welchen bereits zu Beginn des Messzeitraums Daten vorhanden sind.
Die starken Schwankungen beim Kontinent Amerika könnte auch auf die fehlende Unterscheidung zwischen Nord- und Südamerika hindeuten.
Betrachtet man nur den Messzeitraum ab ca. 1870, in dem die Graphen durch konsistente Messungen gleichmäßig verlaufen, fällt trotzdem auf, dass in jedem der Graphen eine Steigung zu erkennen ist. Dies ist eine weiteres Indiz dafür, dass es die Klimaerwärmung tatsächlich gibt und sie zudem alle Bereiche der Erde betrifft und sich nicht auf einzelne Regionen beschränkt.

## <a name="section_af"> Highest BIP Countrys </a>

Mithilfe einer Statistik der Seite Statista konnte herausgefunden werden welche Länder weltweit aktuell das höchste BIP (Bruttoinlandsprodukt) haben (vgl. Quelle 5). Da mithilfe des Datensatzes "GlobalTemperatures" aufgezeigt werden konnte, dass die Klimaerwärmung in einem Zusammenhang mit der Industrialisierung steht und dadurch stark vorangetrieben wird, stellt sich die Frage, ob in Ländern mit einem höheren BIP ein stärkerer Temperaturanstieg zu verzeichnen ist als in Ländern mit einem niedrigeren BIP. Dafür wurden die 5 Länder mit dem aktuell höchsten BIP ausgewählt und mithilfe eines Graphen im Messzeitraum dargestellt.

In [ ]:
germany = globalLandTemperaturesByCountry[globalLandTemperaturesByCountry['Country']=='Germany']
china = globalLandTemperaturesByCountry[globalLandTemperaturesByCountry['Country']=='China']
japan = globalLandTemperaturesByCountry[globalLandTemperaturesByCountry['Country']=='Japan']
unitedStates = globalLandTemperaturesByCountry[globalLandTemperaturesByCountry['Country']=='United States']
unitedKingdom = globalLandTemperaturesByCountry[globalLandTemperaturesByCountry['Country']=='United Kingdom']

germany = germany.groupby('Year')['AverageTemperature'].mean().reset_index()
china = china.groupby('Year')['AverageTemperature'].mean().reset_index()
japan = japan.groupby('Year')['AverageTemperature'].mean().reset_index()
unitedStates = unitedStates.groupby('Year')['AverageTemperature'].mean().reset_index()
unitedKingdom = unitedKingdom.groupby('Year')['AverageTemperature'].mean().reset_index()

sns.lineplot(x="Year",
             y= "AverageTemperature",
             data= germany,
             label = "Germany",
             color = "black").set(title = 'Highes BIP Countrys')

sns.lineplot(x="Year",
             y= "AverageTemperature",
             data= china,
             label = "China",
             color = "red")

sns.lineplot(x="Year",
             y= "AverageTemperature",
             data= japan,
             label = "Japan",
             color = "yellow")

sns.lineplot(x="Year",
             y= "AverageTemperature",
             data= unitedStates,
             label = "United States",
             color = "blue")

sns.lineplot(x="Year",
             y= "AverageTemperature",
             data= unitedKingdom,
             label = "United Kingdom",
             color = "green")

plt.legend()
plt.savefig(r'plots\(19) bipCountrysHigh')

In dieser Grafik ist wie bei der Grafik "Temperaturen pro Kontinent" eine Ungleichmäßigkeit der Graphen zu Beginn des Messzeitraums zu erkennen. Um diese Ungleichmäßigkeiten zu vermeiden wäre es möglich, alle Graphen erst ab einem bestimmten Zeitpunkt anzeigen zu lassen. Es wurde sich jedoch bewusst dagegen entschieden, um die Unterschiedene in den einzelnen Ländern aufzuzeigen und diese später auch mit Ländern mit geringem BIP zu vergleichen.
Es ist besonders auffällig, dass in den europäischen Ländern schon früh Messwerte vorhanden waren. Die Graphen verlaufen zudem sehr gleichmäßig, was für regelmäßige Messungen spricht. Auch in Amerika waren früh Daten vorhanden, diese wurden jedoch nur unregelmäßig erfasst, weshalb der Graph am Anfang sehr unregelmäßig verläuft und erst ab dem Jahre 1820 gleichmäßiger verläuft. Bei den asiatischen Ländern China und Japan wurde zwar erst später, ab dem Jahre 1840 angefangen zu messen, die Graphen verlaufen aber sehr gleichmäßig, was auf eine regelmäßige Messung von Anfang an hindeutet.
Berechnet man den Mittelwert aus dem Beginn der Messzeiträume der einzelnen Länder kommt man mit den abgelesenenn Werten auf das Jahr 1784. Dies ist wichtig um einen Vergleich zu den Ländern mit dem niedrigsten BIP ziehen zu können.
Die Entwicklung der einzelnen Graphen ist untereinander sehr ähnlich und es ist bei jedem der Graphen ein Anstieg zu erkennen.
Um die Entwicklungen besser vergleichen zu können, wird in der nächsten Grafik für jedes der fünf Länder eine Regressionsgerade erzeugt.


## <a name="section_ag"> Highest BIP Countrys Regressionsgeraden</a>

In [ ]:
 sns.regplot(x="Year",
             y= "AverageTemperature",
             data= germany,
             label = "Germany",
             color = "black").set(title = 'Highes BIP Countrys')


sns.regplot(x="Year",
             y= "AverageTemperature",
             data= china,
             label = "China",
             color = "red")

sns.regplot(x="Year",
             y= "AverageTemperature",
             data= japan,
             label = "Japan",
             color = "yellow")

sns.regplot(x="Year",
             y= "AverageTemperature",
             data= unitedStates,
             label = "United States",
             color = "blue")

sns.regplot(x="Year",
             y= "AverageTemperature",
             data= unitedKingdom,
             label = "United Kingdom",
             color = "green")

plt.legend()
plt.savefig(r'plots\(20) bipCountrysHighRegeression')

In dieser Grafik wurde anhand der Messwerte der fünf Länder mit dem höchsten BIP jeweils eine Regressionsgerade erstellt. Anhand dessen lässt sich der bereits in der vorangegangenen Grafik erkannte Temperaturanstieg nochmals verdeutlichen und genauer ablesen.
Verglichen werden die Werte aus dem Jahr 1840 mit den Werten zum Ende des Messzeitraums. Dadurch soll eine bessere Vergleichbarkeit zwischen den Ländern erzeugt werden.
Japan hat im Jahr 1840 eine Durchschnittstemperatur von ca. 11,25° Celsius. Zum Ende des Messzeitraums liegt die Durchschnittstemperatur bei ca. 12,5° Celsius. Der Temperaturanstieg liegt bei +1,25° Celsius.
China hat im Jahr 1840 eine Durchschnittstemperatur von ca. 6,00° Celsius. Zum Ende des Messzeitraums liegt die Durchschnittstemperatur bei ca. 7,5° Celsius. Der Temperaturanstieg liegt bei +1,5° Celsius.
Deutschland hat im Jahr 1840 eine Durchschnittstemperatur von ca. 7,75° Celsius. Zum Ende des Messzeitraums liegt die Durchschnittstemperatur bei ca. 8,75° Celsius. Der Temperaturanstieg liegt bei +1,00° Celsius.
United Kingdom hat im Jahr 1840 eine Durchschnittstemperatur von ca. 8,25° Celsius. Zum Ende des Messzeitraums liegt die Durchschnittstemperatur bei ca. 9,00° Celsius. Der Temperaturanstieg liegt bei +0,75° Celsius.
United States hat im Jahr 1840 eine Durchschnittstemperatur von ca. 7,6° Celsius. Zum Ende des Messzeitraums liegt die Durchschnittstemperatur bei ca. 9,9° Celsius. Der Temperaturanstieg liegt bei +2,3° Celsius.
Die werte konnten nur schätzungweise aus der Grafik abgelesen werden.
Der Mittelwert des Temperaturanstiegs der fünf Länder mit dem höchsten BIP liegt bei 1,36. Nach Betrachtung der Temperaturentwicklung der Länder mit dem niedrigsten BIP kann daraus ein Schluss gezogen werden.

## <a name="section_ah"> Lowest BIP Countrys</a>
Aus einer Statistik der Seite Statista lässt sich ablesen, welche Länder die 5 Länder mit dem aktuell weltweit niedrigsten BIP (Bruttoinlandsprodukt) sind (vgl. Quelle 6).
Auf Platz zwei der Länder mit dem niedrigsten BIP liegt Südsudan. Da für Südsudan im Datensatz jedoch keine Werte vorliegen, wird es in der nachfolgenden Grafik nicht mitaufgelistet.
Allgemein lässt sich beobachten, dass es sich bei den Ländern mit dem weltweit niedrigsten BIP um Länder handelt deren Durchschnittstemperatur deutlich über der Durchschnittstemperatur der Länder mit dem weltweit höchsten BIP liegen.

In [ ]:
burundi = globalLandTemperaturesByCountry[globalLandTemperaturesByCountry['Country']=='Burundi']
sierraleone = globalLandTemperaturesByCountry[globalLandTemperaturesByCountry['Country']=='Sierra Leone']
somalia = globalLandTemperaturesByCountry[globalLandTemperaturesByCountry['Country']=='Somalia']
mozambique = globalLandTemperaturesByCountry[globalLandTemperaturesByCountry['Country']=='Mozambique']
madagascar = globalLandTemperaturesByCountry[globalLandTemperaturesByCountry['Country']=='Madagascar']

burundi = burundi.groupby('Year')['AverageTemperature'].mean().reset_index()
sierraleone = sierraleone.groupby('Year')['AverageTemperature'].mean().reset_index()
somalia = somalia.groupby('Year')['AverageTemperature'].mean().reset_index()
mozambique = mozambique.groupby('Year')['AverageTemperature'].mean().reset_index()
madagascar = madagascar.groupby('Year')['AverageTemperature'].mean().reset_index()

sns.lineplot(x="Year",
             y= "AverageTemperature",
             data= burundi,
             label = "Burundi",
             color = "black").set(title = 'Lowest BIP Countrys')

sns.lineplot(x="Year",
             y= "AverageTemperature",
             data= sierraleone,
             label = "Sierra Leone",
             color = "red")

sns.lineplot(x="Year",
             y= "AverageTemperature",
             data= somalia,
             label = "Somalia",
             color = "yellow")

sns.lineplot(x="Year",
             y= "AverageTemperature",
             data= mozambique,
             label = "Mozambique",
             color = "blue")

sns.lineplot(x="Year",
             y= "AverageTemperature",
             data= madagascar,
             label = "Madagascar",
             color = "green")

plt.legend()
plt.savefig(r'plots\(21) bipCountrysLow')

In dieser Grafik sind die fünf zuvor ausgewählten Länder als Graph dargestellt. Es handelt sich hierbei um die Länder Burundi, Sierra Leone, Somalia, Mozambique und Madagascar. Auffällig ist, dass der Graph aller Länder bis auf Madagascar erst im Jahre 1850 beginnt. Daraus lässt sich schließen, dass für diese Länder bis zum Jahre 1850 keine Messwerte vorliegen. Für das Land Madagascar liegen schon vor 1800 Messwerte vor. Der Graph ist jedoch bis ca. 1890 sehr unregelmäßig, was darauf hindeutet, dass bis zu diesem Zeitpunkt nur unregelmäßige Messungen durchgeführt wurden. Auch die Graphen der anderen Ländern gewinnen erst ab ca. 1870 an Regelmäßigkeit, was für regelmäßige Messungen spricht. Der Graph von Sierra Leone weißt bereits von Beginn an einen hohen Grad an Gleichmäßigkeit auf.
In jedem der Graphen ist eine eindeutige Steigung zu sehen. Hierauf wird jedoch in der folgenden Grafik noch weiter eingegangen.
Der Mittelwert des Beginns der Messzeiträume der einzelnen Länder liegt, sofern man ihn mit den abgelesenen Werten berechnet, bei 1826. Im Vergleich zu den Ländern mit dem höchsten BIP macht das einen Unterschied von 42 Jahren aus. Dies bedeutet, dass in den fünf Ländern mit dem höchsten BIP durchschnittlich 42 Jahre früher mit den Aufzeichnungen der Messwerte begonnen wurde.
In der folgenden Grafik wird mithilfe von Regressionsgeraden noch auf den genauen Temperaturverlauf der fünf Länder mit dem niedrigsten BIP eingegangen.


## <a name="section_ai"> Lowest BIP Countrys Regressionsgeraden </a>

In [ ]:
sns.regplot(x="Year",
             y= "AverageTemperature",
             data= burundi,
             label = "Burundi",
             color = "black").set(title = 'Lowest BIP Countrys')

sns.regplot(x="Year",
             y= "AverageTemperature",
             data= sierraleone,
             label = "Sierra Leone",
             color = "red")

sns.regplot(x="Year",
             y= "AverageTemperature",
             data= somalia,
             label = "Somalia",
             color = "yellow")

sns.regplot(x="Year",
             y= "AverageTemperature",
             data= mozambique,
             label = "Mozambique",
             color = "blue")

sns.regplot(x="Year",
             y= "AverageTemperature",
             data= madagascar,
             label = "Madagascar",
             color = "green")

plt.legend()
plt.savefig(r'plots\(22) bipCountrysLowRegeression')

In dieser Grafik wurde anhand der Messwerte der fünf Länder mit dem niedrigsten BIP für jedes der Länder jeweils eine Regressionsgerade erstellt. Um den Temperaturanstieg bewerten zu können wird für jedes Land ein Wert des Jahres 1850 und ein Wert zum Ende des Messzeitraums schätzungsweise aus der Grafik abgelesen. Es wurde das Jahr 1850 ausgewählt, da dort der Messzeitraum von vier der fünf Länder beginnt und die beste Vergleichbarkeit gegeben ist, wenn man einen einheitlichen Anfangspunkt auswählt.
Somalia hat im Jahr 1850 eine Durchschnittstemperatur von ca. 26° Celsius. Zum Ende des Messzeitraums liegt die Durchschnittstemperatur bei ca. 27,5° Celsius. Der Temperaturanstieg liegt bei +1,5° Celsius.
Sierra Leone hat im Jahr 1850 eine Durchschnittstemperatur von ca. 25,25° Celsius. Zum Ende des Messzeitraums liegt die Durchschnittstemperatur bei ca. 26,5° Celsius. Der Temperaturanstieg liegt bei +1,25° Celsius.
Mozambique hat im Jahr 1850 eine Durchschnittstemperatur von ca. 22,8° Celsius. Zum Ende des Messzeitraums liegt die Durchschnittstemperatur bei ca. 24,2° Celsius. Der Temperaturanstieg liegt bei +1,40° Celsius.
Madagascar hat im Jahr 1850 eine Durchschnittstemperatur von ca. 22,2° Celsius. Zum Ende des Messzeitraums liegt die Durchschnittstemperatur bei ca. 23,3° Celsius. Der Temperaturanstieg liegt bei +1,10° Celsius.
Burundi hat im Jahr 1850 eine Durchschnittstemperatur von ca. 19,5° Celsius. Zum Ende des Messzeitraums liegt die Durchschnittstemperatur bei ca. 20,5° Celsius. Der Temperaturanstieg liegt bei +1,00° Celsius.
Der Mittelwert des Temperaturanstiegs der fünf Länder mit dem höchsten BIP liegt bei +1,25° Celsius. Im Vergleich zum Temperaturanstieg in den fünf Ländern mit dem höchsten BIP liegt der Temperaturanstieg in den Ländern mit dem niedrigsten BIP 0,11° Celsius darunter.
Dadurch, dass lokale Temperaturveränderungen so viele Einflussfaktoren haben, lässt sich die Annahme, dass in Ländern mit hohem BIP allgemein eine höhere Temperatursteigung vorliegt als in Ländern mit geringem BIP, trotz des Ergebnisses nicht vollständig bestätigen. Das Ergebnis dieses Vergleichs zwischen fünf Ländern mit hohem BIP und fünf Ländern mit niedrigem BIP ist nur ein Indiz dafür und die Annahme müsste weiter untersucht werden.


# <a name="section_ba"> Verlauf der CO2-Werte </a>

## <a name="section_bb"> Beschreibung des Datensatzes </a>
Im Datensatz "GlobalCo2Values" werden vom Jahr 1958 bis 2017 verschiedene Werte aufgelistet.
Für jedes Jahr wurden monatlich Werte gemessen, welche den Kohlenstoffdioxidgehalt der Atmosphäre darstellen. Zu den gemessenen Attributen zählen:
Carbon Dioxide (ppm) - Dieser Wert beschreibt den Kohlenstoffdioxidgehalt in der Atmosphäre.
Seasonally Adjusted CO2 (ppm) - Dieser Wert beschreibt den Kohlestoffdioxidgehalt nach einer saisonalen Glättung des Wertes. Das bedeutet, dass die natürliche saisonale Schwankung der Kohlenstoffdioxidkonzentration der Atmosphäre herausgerechnet wird.
Carbon Dioxide Fit (ppm) - Dieser Wert beschreibt den Kohlenstoffdioxidgehalt der Atmosphäre, nachdem einige Funktionen angewandt wurden, um die Linearität der Werte zu erhöhen.
Seasonally Adjusted CO2 Fit (ppm) - Dieser Wert beschreibt den Kohlestoffdioxidgehalt der Atmosphäre nach saisonaler Glättung und Anwendung von Funktionen zur Erhöhung der Linearität.
Da der Kohlenstoffdioxidgehalt der Atmosphäre den Hauptfaktor für die globale Klimaerwärmung darstellt und durch die Industrialisierung vermehrt erzeugt wird, ist es interessant zu sehen, ob zwischen den CO2-Werten und den Werten der Durchschnittstemperatur eine Korrelation besteht.

In [ ]:
globalCo2Values = pd.read_csv( Path().joinpath('data', 'GlobalCo2Values.csv'))
globalCo2neural= globalCo2Values.copy()
globalCo2Values.isnull().sum()

Diese Tabelle zeigt die Anzahl der Nullwerte in den einzelnen Spalten. Es handelt sich hierbei jedoch um eine sehr geringen, zu vernachlässigbare Anzahl.

In [ ]:
nasPerDate = globalCo2Values.set_index('Decimal Date')['Carbon Dioxide (ppm)'].isnull()\
    .reset_index()\
    .groupby('Decimal Date')\
    .sum()

nasPerDate.columns = ['Missing values']

nasPerDate.head(20)

In [ ]:
sns.lineplot(x="Decimal Date",
             y= "Missing values",
             data= nasPerDate).set(title = 'Nas pro Jahr')
plt.savefig(r'plots\(23) nasPerDateCo2')

In dieser Grafik sind die Nullwerte der Spalte "Carbon Dioxide (ppm)" über den Zeitverlauf dargestellt. Insgesamt gibt es in dem Datensatz in der Spalte "Carbon Dioxide (ppm)" nur 17 Nullwerte. Man sieht, dass das Auftreten, des Nullwerte direkt zu Beginn der Messungen deutlich häufiger vorkommt. Dadurch, dass es sich jedoch nur eine sehr geringe Anzahl an Nullwerten handelt ist die Aussagekraft des Auftretens sehr gering. Vermutlich wurden von Beginn an sehr gute Messverfahren angewandt, welche konsistent die Werte gemessen haben.

In [ ]:
mergedCoAndGlobalTemps = pd.merge(globalCo2Values, globalTemperatures, how='left', on=["Year","Month"])

mergedCoAndGlobalTemps.head()

Diese Tabelle zeigt die oberen fünf Zeilen einer neu erstellten Tabelle. Hierzu wurden die Daten der Datensätze "GlobalCo2Values" und "GlobalTemperature" gemerged. Dies ist im weiteren Verlauf wichtig, da dadurch Korrelationen zwischen den Werten erkannt werden können. Dadurch kann die These "Hat die Industrialisierung einen Einfluss auf die Klimaerwärmung?" weiter untermauert bzw. widerlegt werden.

## <a name="section_bc"> Korrelationskoeffizient zwischen "GlobalLandTemperatures" und "GlobalCo2Values" </a>

In [ ]:
mergedCoAndGlobalTemps.corr()

Vorliegende Tabelle zeigt die Korrelationskoeffizienten zwischen den einzelnen Attributen des Datensatzes "GlobalTemperatures" und den Attributen des Datensatzes "GlobalCo2Values" an. Der Korrelationskoeffizient zeigt die lineare Stärke einer Beziehung zwischen zwei Variablen. Je näher sich der Korrelationskoeffizient an eins annähert, desto stärker ist der Zusammenhang zwischen den zwei Variablen. Besonders interessant sind hierbei die Korreltaionskoeffizienten zwischen den Variablen der verschiedenen Datensätze.
Betrachtet man den Wert "Carbon Dioxide (ppm)" im Vergleich zur "LandAverageTemperature" sieht man, dass eine positive Korrelation von 0,087895 besteht. Betrachtet man die Korrelation zur "LandAndOceanTemperature" ist der Korrelationkoeffizient noch höher. Hier liegt er bei 0,177141. Da man erst ab einem Wert von 0,2 von einer schwachen Korrelation spricht, liegt hier keine eindeutige Korrelation vor. Zwischen den Variablen der Datensätze "GlobalTemperatures" und "GlobalCo2Values" besteht somit laut dem Korrelationkoeffizient keine Korrelation. Die Korrelation der Datensätze soll anhand der nächsten Grafik weiter überprüft werden.

## <a name="section_bd"> Pairplot von "GlobalTemperatures" und "GlobalCo2Values" </a>

In [ ]:
mergedCoAndGlobalTempsWitUncertaintValuesRemoved = mergedCoAndGlobalTemps.drop(['Decimal Date',
                                                                                  'LandAverageTemperatureUncertainty',
                                                                                  'LandMaxTemperatureUncertainty',
                                                                                  'LandMinTemperatureUncertainty',
                                                                                  'LandAndOceanAverageTemperatureUncertainty'], axis=1)


sns.pairplot(mergedCoAndGlobalTempsWitUncertaintValuesRemoved)
plt.savefig(r'plots\(24) mergedPairplot')

In vorliegender Grafik sind die Korrelationen zwischen den Datensätzen "GlobalTemperatures" und "GlobalCo2Values" noch einmal graphisch dargestellt. Der Fokus liegt auch hier auf dem Vergleich der beiden Datensätze untereinander.
Vergleicht man das Attribut "LandAverageTemperature" aus dem Datensatz "GlobalLandTemperatures" mit den Attributen "Carbon Dioxide (ppm)", "Seasonally Adjusted CO2(ppm)", "Carbon Dioxide Fit (ppm)" und "Seasonally Adjusted CO2 Fit (ppm)" sieht man, dass nur eine sehr schwache Korrelation zu erkennen ist. Die Werte sind in 5 Bereiche unterteilt. Diese steigen an, je höher der Co2-Wert auf der x-Achse ist. Dieser Verlauf ist bei jedem der vier Attribute "LandAverageTemperature", "LandMaxTemperature", "LandMinTemperature" und "LandAndOceanAverageTemperature" des Datensatzes GlobalTemperatures zu sehen, wenn man die Korrelation mit den vier genannten Attributen des Datensatzes "GlobalCo2Values" betrachtet. Es lässt sich jedoch nur auf eine sehr schwache Korrelation schließen. Eine starke Korrelation sieht man, wenn man die Korrelation zwischen der "LandAverageTemperature" und der "LandMaxTemeperature" betrachtet. Dort entsteht eine lineare Gerade, welche für eine sehr starke Korrelation spricht.

## <a name="section_be"> 3D Scatterplot mit "LandAverageTemperature", "Carbon Dioxide (ppm)" und Year </a>

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter3D(mergedCoAndGlobalTemps['LandAverageTemperature'], mergedCoAndGlobalTemps['Carbon Dioxide (ppm)'], mergedCoAndGlobalTemps['Year'], c='skyblue', s=60)
plt.savefig(r'plots\(25) combinedYearLandAverageAndCarbon')

Dieser 3D Scatterplot zeigt das Verhältnis zwischen dem Jahr, dem Wert der "LandAverageTemperature" aus dem Datensatz "GlobalTemperatures" und dem Wert "Carbon Dioxide (ppm)" aus dem Datensatz "GlobalCo2Values". Die z-Achse steht für das jeweilige Jahr. Die x-Achse für die Temperatur und die y-Achse für den Wert "Carbon Dioxide (ppm)".
Aufgrund der Dreidimensionalität ist es jedoch schwierig die Zusammenhänge zwischen den einzelnen Variablen zu erkennen. Es ist jedoch ersichtlich, dass auch hier keine starke Korrelation vorliegt, da kein linear ansteigender Graph erzeugt wird.

# <a name="section_c"> Fazit</a>
In vorliegender Arbeit konnte viel zum Temperaturverlauf der Erde und dessen Entwicklung, sowie die Entwicklung der Co2-Werte erforscht werden. Auf Grundlage dessen sollen nun die zu Beginn aufgestellten Hypothesen beantwortet werden.
1. Gibt es die Klimaerwärmung wirklich?
In den Grafiken, welche zur Analyse des Temperaturverlaufs genutzt wurden, war im Laufe der Zeit stets ein Temperaturanstieg sichtbar. Im Algorithmus zur Datenvorhersage, wurde dieser Temperaturanstieg nochmals besonders deutlich, da er sich auch bis zum prognostizierten Jahre 2030 positiv entwickelt. Unklar ist jedoch, ob es sich dabei um eine dauerhafte Klimaerwärmung handelt, oder ob dies bloß eine natürliche Schwankung der Erdtemperatur ist. Um das zu überprüfen, hätte eine Langzeitbetrachtung der Erdtemperatur erfolgen müssen.
2. Hat die Industrialisierung einen Einfluss auf die Klimaerwärmung?
Anhand verschiedener Grafiken, wurde deutlich, dass die Erdtemperatur besonders ab dem Jahre 1970 ansteigt, in welchem die dritte Industrielle Revolution stattfand. Dies spricht deutlich dafür, dass die Industrialisierung einen Einfluss auf die Klimaerwärmung hat. Zwischen dem Temperaturverlauf der Erde und dem ausgestoßenen Co2, welches in großen Mengen durch die Industrie erzeugt wird, war jedoch keine bis sehr schwache Korrelation sichtbar. Dies spricht gegen den Zusammenhang zwischen Industrialisierung und der Klimaerwärmung. Da der deutliche Temperaturanstieg ab dem Jahr 1970 jedoch nicht zu vernachlässigen ist, ist es wahrscheinlich, dass neben dem ausgestoßenen Co2 weitere Aspekte der Industrialisierung einen Einfluss auf die Klimaerwärmung haben und somit der Zusammenhang zwischen Industrialisierung und Klimaerwärmung besteht. Um dies zu validieren, müssten jedoch weitere Korrelationen zwischen dem Temperaturverlauf der Erde und weiteren Einflussfaktoren, die die Industrialisierung mit sich bringt, überprüft werden.
3. Wie wird sich der Temperaturverlauf bis zum Jahr 2030 entwickeln?
Laut einem Algorithmus zur Datenprognose, welcher auf den Daten des Datensatzes "GlobalTemperatures" beruht, wird die durchschnittliche Erdtemperatur bis zum Jahr 2030 knapp über 10°Celsius liegen. Somit liegt sie zwischen 1-1,5°Celsius über dem Höchstwert zu Beginn der Aufzeichnungen im Jahr 1750.

Vergleicht man die Temperaturdaten mit anderen Quellen, fällt auf, dass die Daten des verwendeten Datensatzes im Durchschnitt darunter liegen. Der Grund hierfür ist jedoch nicht ohne Weiteres zu bestimmen.

# <a name="section_d"> Dataset Quellen </a>
https://www.kaggle.com/datasets/ucsandiego/carbon-dioxide <br>
https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data <br>
https://www.kaggle.com/datasets/andradaolteanu/country-mapping-iso-continent-region


# <a name="section_e"> Quellverzeichnis </a>
Quelle 1: https://wetter.tagesschau.de/wetterthema/2021/11/10/zur-messung-der-lufttemperatur.html <br>
Quelle 2: https://www.geodatenmanufaktur.de/howtos/wassertemperatur.html <br>
Quelle 3: https://studyflix.de/statistik/lineare-regression-2147 <br>
Quelle 4: https://www.schubu.at/p53/klima <br>
Quelle 5: https://de.statista.com/statistik/daten/studie/157841/umfrage/ranking-der-20-laender-mit-dem-groessten-bruttoinlandsprodukt/#:~:text=Mit%20einem%20Bruttoinlandsprodukt%20(BIP)%20von,Abstand%20auf%20dem%20zweiten%20Platz <br>
Quelle 6: https://de.statista.com/statistik/daten/studie/247140/umfrage/laender-mit-dem-niedrigsten-bruttoinlandsprodukt-bip-pro-kopf/#:~:text=Rangliste%20der%20Länder%20mit%20dem%20niedrigsten%20Bruttoinlandsprodukt%20(BIP)%20pro%20Kopf%202021&text=Das%20ostafrikanische%20Land%20Burundi%20ist, niedrigsten%20Bruttoinlandsprodukt%20pro%20Kopf%20weltweit <br>
Quelle 7:  https://www.statsoft.de/glossary/R/RootMeanSquareRMSError.htm#:~:text=Um%20den%20RMS%2D(root,die%20den%20gesamten%20Fehler%20beschreibt. <br>
Quelle 8: https://www.care.de/schwerpunkte/klimakrise/klimawandel/#c2036 <br>

# Quellverzeichnis
https://wetter.tagesschau.de/wetterthema/2021/11/10/zur-messung-der-lufttemperatur.html
https://www.geodatenmanufaktur.de/howtos/wassertemperatur.html
https://studyflix.de/statistik/lineare-regression-2147
https://www.schubu.at/p53/klima
https://de.statista.com/statistik/daten/studie/157841/umfrage/ranking-der-20-laender-mit-dem-groessten-bruttoinlandsprodukt/#:~:text=Mit%20einem%20Bruttoinlandsprodukt%20(BIP)%20von,Abstand%20auf%20dem%20zweiten%20Platz
https://de.statista.com/statistik/daten/studie/247140/umfrage/laender-mit-dem-niedrigsten-bruttoinlandsprodukt-bip-pro-kopf/#:~:text=Rangliste%20der%20Länder%20mit%20dem%20niedrigsten%20Bruttoinlandsprodukt%20(BIP)%20pro%20Kopf%202021&text=Das%20ostafrikanische%20Land%20Burundi%20ist,niedrigsten%20Bruttoinlandsprodukt%20pro%20Kopf%20weltweit
